In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds

# Завантаження та підготовка даних
(train_ds, test_ds), metadata = tfds.load(
    'eurosat/rgb',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True
)

num_classes = metadata.features['label'].num_classes

IMG_SIZE = 128
BATCH_SIZE = 32

def normalize_img(image, label):
    """Функція для нормалізації зображень"""
    return tf.cast(image, tf.float32) / 255., label

# Нормалізація даних та створення пакетів даних
train_ds = train_ds.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, (128, 128)), y))
train_ds = train_ds.cache()
train_ds = train_ds.shuffle(metadata.splits['train'].num_examples)
train_ds = train_ds.batch(BATCH_SIZE)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

test_ds = test_ds.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, (128, 128)), y))
test_ds = test_ds.batch(BATCH_SIZE)
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)

# Створення згорткової нейронної мережі
model = tf.keras.Sequential([
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])

# Компіляція моделі
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Навчання моделі
epochs=10
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=epochs
)

# Оцінка результатів
test_loss, test_acc = model.evaluate(test_ds, verbose=2)
print('Test accuracy:', test_acc)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/eurosat/rgb/2.0.0.incomplete110OXS/eurosat-train.tfrecord*...:   0%|      …

Dataset eurosat downloaded and prepared to /root/tensorflow_datasets/eurosat/rgb/2.0.0. Subsequent calls will reuse this data.
Epoch 1/10
675/675 [==============================] - 410s 593ms/step - loss: 2.2963 - accuracy: 0.1076 - val_loss: 2.2962 - val_accuracy: 0.1065
Epoch 2/10
675/675 [==============================] - 414s 613ms/step - loss: 2.2956 - accuracy: 0.1069 - val_loss: 2.2956 - val_accuracy: 0.1065
Epoch 3/10
675/675 [==============================] - 391s 578ms/step - loss: 2.2951 - accuracy: 0.1098 - val_loss: 2.2954 - val_accuracy: 0.1065
Epoch 4/10
675/675 [==============================] - 402s 595ms/step - loss: 2.2951 - accuracy: 0.1098 - val_loss: 2.2955 - val_accuracy: 0.1061
Epoch 5/10
675/675 [==============================] - 401s 594ms/step - loss: 2.2951 - accuracy: 0.1090 - val_loss: 2.2955 - val_accuracy: 0.1061
Epoch 6/10
675/675 [==============================] - 398s 590ms/step - loss: 2.2951 - accuracy: 0.1106 - val_loss: 2.2953 - val_accuracy: 0.11